# PRIMMDebug Student Survey Analysis

This notebook contains a narrative of the code used to analyse the responses from the student survey taken after usage of PRIMMDebug. The goal of the survey was to understand students' perspectives towards PRIMMDebug, which could be triangulated with their actions within the PRIMMDebug tool.

The analysis is split up into several sections:
1. Summary statistics across the whole dataset.
2. Factor analysis
3. (Potentially) Alternative analysis
4. An appendix of visualisations for each individual question response.

First, let us import the necessary libraries and data.

In [ ]:
suppressPackageStartupMessages(library(ltm))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(forcats))
suppressPackageStartupMessages(library("plotly"))

source("data_cleaning.r")
survey_response_data <- get_survey_data_numerical()
all_quant_responses <- names(survey_response_data)[sapply(survey_response_data, is.numeric)]

source("constants.r")
question_labels <- get_question_labels()


primmdebug_usability_questions <- c("Q1_1","Q1_2","Q1_3","Q1_4","Q1_5")
restrictive_factors_questions <- c("Q2_1", "Q2_2", "Q2_3","Q2_4")
primmdebug_challenge_questions <- c("Q2_5","Q3_1","Q3_2","Q4")
sifft_utility_questions <- c("Q5","Q6")
likert_response_questions <- c(primmdebug_usability_questions, restrictive_factors_questions, primmdebug_challenge_questions, sifft_utility_questions) #TODO: Not sure where to put this

## 1. Summary Statistics
We first report on overall statistics of the survey to give context to the responses.

In [ ]:
source("constants.r")

paste("Number of student respondents: ", get_total_student_count())

gender_count <- plot_ly(
    x = get_gender_counts()$gender,
    y = get_gender_counts()$count,
    type = "bar"
) %>% layout(title = "Gender split of participating students")
gender_count

year_group_count <- plot_ly(
    x = get_year_group_split()$year_group,
    y = get_year_group_split()$count,
    type = "bar"
) %>% layout(title = "Year group of participating students")
year_group_count

school_split <- plot_ly(
    x = get_school_split()$school,
    y = get_school_split()$count,
    type = "bar"
) %>% layout(title = "School split of participating students")
school_split


## 2. Factor Analysis
Factor analysis was performed with the intention of investigating correlations between different groups of survey items (treated as latent factors), in order to uncover general patterns in responses to the survey items.

The original intention was to perform confirmatory factor analysis (CFA) to identify the fitness of these factors and then if able, assess the correlations between them. However, Leonie et al., (2024), as well as other FA guides, suggests that this was not appropriate for out study for several reasons:
- The factors used in CFA should be "based on strong a-priori hypotheses" (Leonie et al., 2024). We were not confident that the groupings of these hypotheses were strong enough for suitable confirmatory analysis. Combined with the relatively small number of student participants on which to perform factor analyis,
- Factors could also be based on underlying theory or existing survey instruments. However, our survey was originally developed which is less suited to CFA given the lack of validation performed on it.

The following steps document each step of the factor analysis we performed.

### a. Transformation
Since survey items use different Likert response options, the data needs to be transformed (Leonie et al., 2024). This was done by:
- Normalising data to have the same min and max values. Given the closeness in range of Likert scale used (as well as the non-normal distributions of some responses), this was more appropriate than transformation.
  - The `datawizard` package Patil (2022) was used to do this.
- Reversing code negatively worded items? Will be able to tell from correlation
(Can try the effect of standardisation on the output of the FA but it intuitively seems far less appropriate.)

In [ ]:
library("datawizard")
normalised_likert_responses <- rescale(survey_response_data[, likert_response_questions], to = c(1, 5))

### b. Safety Checks
To verify whether EFA could be conducted, we performed the following safety checks, as discussed in Cohen et al., (2018) and Leonie et al., (2024):
- A Bartlett's test of sphericity was performed to verify whether there are sufficiently large correlations between the survey items to warrant factor analysis. In order to perform FA, **a significant result (i.e., p < 0.05) was required** (Humble 2020, Leonie et al., 2024).
- A Kaiser-Meyer-Olkin test was performed to measure the amount of common variance items among the survey items. According to Kaiser (1970, as cited in Humble, 2020) and Hutcheson and Sofroniou (1999, as cited in Humble, 2020), **we wanted a value above 0.7**.
- The normality of the distributions was assessed through a visual inspection of the responses and measures of skewness/kurtosis.
  - If data is not normally distributed, "an estimation method that is robust against non-normality should be used" (Leonie et al., 2024) - different type of estimator/extraction in factor analysis function

Other tests to iron out:
- Is the sample size sufficiently large?
  - Given there are (n survey items reliable) and other items, how many participants is suitable?

Note that responses to each survey item was compulsory, therefore there were no missing items in the matrix of responses.

The correlation matrix used for the rest of the code is a *polychloric* matrix as we are dealing with ordinal data of varying scales (which has been normalised). Not only is this more appropriate, but ``the polychoric method of estimating correlation is more robust to the violation of normality assumption and could be preferably used, compared to the Pearson correlation, when data is not approximately normal and of ordinal level'' (p. 32, Emanuelsson, 2021). This is fitting given the non-normal distribution of some of the survey responses.

In [ ]:
suppressPackageStartupMessages(library("psych"))
source("analysis.r")

polychoric_matrix <- polychoric_correlation_matrix(normalised_likert_responses)

cat("Cronbach's alpha across questions:", internal_consistency(normalised_likert_responses), "\n")

cat("p-value from barlett test:", round(cortest.bartlett(polychoric_matrix, n = get_total_student_count())$p.value, 3), "\n") #TODO:Get correlation matrix to pass in here?

cat("Overall MSA for likert scale responses in survey", KMO(normalised_likert_responses)$MSA,"\n")


### c. Extracting Factors

If the safety checks in step (b) indicate factor analysis can be performed, we first need to determine how many factors we want to use. This is done through the following steps:
1. A "total variance explained" table and Scree plot were generated.
2. The eigenvalues and the Scree were plot inspected. These, along with the following considerations, were used to determine the number of factors:
   - The number of factors suggested by parallel analysis.
   - The Kaiser criterion - retain eigenvalues greater than one (Kaiser, 1970)
   - The factors chosen should explain over 50% of the variance.
3. Once the number of factors were decided, a component matrix was generated using the following parameters:
   - Extraction/estimation method:
      - Started with principal axis factoring as this is seemingly the most popular estimation method for non-normal distributed underlying data.
      - This had to be done using the `fa` function in the `psych` package, which had the added bonus of being able to pass in my own polychloric matrix.
      - Depending on results of this, explore with robust maximum likelihood (MLR, as used in Leonie et al., (2024)).
      - Then, if FA is applicable, find out what other methods do not require the assumption of normal distribution.
   - Rotation method: We were specifically interested in investigating the correlations between the factors; this was the entire goal of performing EFA. As a result, an oblique rotation method was entirely appropriate and recommended by Watkins (2018). We first tried **promax** and then **oblimin** due to their popularity and their general recommendation in Watkins (2018).
These first three steps were replicated by a research colleague.
1. The simplicity (where variables strongly/solely load onto a single factor, ideally with loadings of > 0.3) and the interpretability/logic of the factors (whether the survey items loaded onto a factor are logical) were consulted to decide the suitability of the model. If not suitable, the following parameters could be tuned:
   - Number of factors in model.
   - The rotation method.
   - The extraction method.
This exploration should help to inform the method of estimation and rotation to use for the EFA model created.

If applicable, the final factor parameters are reported below:

In [ ]:
suppressPackageStartupMessages(library(lavaan))
suppressPackageStartupMessages(library(plotly))
source("analysis.r")

eigenvalue <- eigen(polychoric_matrix)$values
#eigen function only returns a vector of eigenvalues, so this is converted to a matrix with percentages/cumulative percentages
variance_vector <- data.frame(eigenvalue)
variance_vector$percent <- prop.table(variance_vector$eigenvalue) * 100
variance_vector$cumulative_percent <- (cumsum(eigenvalue) / sum(eigenvalue)) * 100

fig <- plot_ly(
    type = "table",
    columnwidth = c(100, 100, 100),
    columnorder = c(0, 1, 2),
    header = list(
        values = colnames(variance_vector),
        align = c("center", "center", "center"),
        line = list(width = 1, color = 'black'),
        fill = list(color = c("grey", "grey", "grey")),
        font = list(family = "Arial", size = 14, color = "white")
    ),
    cells = list(
        values = t(variance_vector),
        align = c("center", "center", "center"),
        line = list(color = "black", width = 1)
    )
) %>%
layout(title = "Total variance explained - initial eigenvalues")
fig

fa.parallel(normalised_likert_responses, fa="fa", main="Scree plot for factor analysis")

efa_model <- fa(polychoric_matrix, nfactors = 2, fm = "pa", rotate = "promax", n.obs = get_total_student_count())

### d. Inter-Factor Correlations
If step (c) was completed, the interfactor correlation matrix was inspected using the `Phi` value of the `fa()` function, inspecting responses to individual items where necessary.. In the unlikely event that the correlation matrix has more than 20 items, some correction will be performed.


In [ ]:
print(efa_model$Phi)

## 3. Alternative Extensions

If factor analysis could not be performed, we planned to instead conduct some less advanced analysis inspired by visual inspection of the survey responses. This includes the following ideas:

- Usability
  - Calculate a mini-SUS score for the 5 survey items
- SIFFT process
  - How did usefulness of SIFFT process correlate with students' frequency of SIFFT?
- Restrictive factors
  - Correlational analysis (and internal consistency) on restrictive factors of PRIMMDebug (perhaps minus the find the error score)
  - Perform correlation of this with gender
- Correlational analysis between restrictive factors and utility of SIFFT (as these had the most negative responses)
- General correlational analysis considerations
  - Is there some way of aggregating a standardised score for each component of the survey?
  - PCA with normalisation?
  - Or create normalised Z-score? The higher the Z score, the more positive the perspectives
    - Could these, or do some correlational analysis with gender
- Support findings with short qual responses
- Relation to student log data e.g. number of uses/challenges attempted?
  - How did responses to restrictive factors (again, a score would be helpful), relate to the quality of students' reflections in PRIMMDebug?

## References
Louis Cohen, Lawrence Manion, and Keith Morrison. 2017. Factor analysis, cluster analysis and structural equation modelling. In Research Methods in Education (8 ed.). Routledge, New York, Chapter 40, 753–775. https://doi.org/10.4324/9781315456539-43 \
Humble, S. (2020). Quantitative Analysis of Questionnaires: Techniques to Explore Structures and Relationships (1st ed.). Routledge. https://doi.org/10.4324/9780429400469 \
Hutcheson, G. D. (1999).  The multivariate social scientist. SAGE Publications, Ltd., https://doi.org/10.4135/9780857028075 \
Kaiser, H. F. (1970). A second generation little jiffy. Psychometrika, 35(4), 401–415. https://doi.org/10.1007/BF02291817
Leonie V.D.E. Vogelsmeier, Mohammed Saqr, Sonsoles López-Pernas, Joran Jongerling (2024). Factor Analysis in Education Research Using R. In M. Saqr & S. López-Pernas (Eds.), Learning analytics methods and tutorials: A practical guide using R (pp. 673-703).Springer, Cham. doi: [10.1007/978-3-031-54464-4_20](https://link.springer.com/chapter/10.1007/978-3-031-54464-4_20) \
Patil I, Makowski D, Ben-Shachar MS, Wiernik BM, Bacher E, Lüdecke D (2022) Datawizard: An R package for easy data preparation and statistical transformations. J Open Source Softw 7:4684. https://doi.org/10.21105/joss.04684 \
Watkins, M. W. (2018). Exploratory Factor Analysis: A Guide to Best Practice. Journal of Black Psychology, 44(3), 219-246. https://doi.org/10.1177/0095798418771807 (Original work published 2018)

## Appendix A. Visualisation of Individual Survey Item Responses
We now provide some basic plots of the responses to individual Likert response items for comprehensiveness. Each question contains information on:
- The normality of the distribution.
- The skewness and kurtosis of the distribution.

In [ ]:
library(moments)

#Q1 responses
question_1_items <- c("Q1_1", "Q1_2", "Q1_3", "Q1_4", "Q1_5")

fig_list <- list()
for (question in question_1_items) {
    cat(question_labels[question],":\n")
    cat("Skewness: ", round(skewness(table(factor(survey_response_data[, question], levels = 1:5))), 2), "\n")
    cat("Kurtosis: ", round(kurtosis(table(factor(survey_response_data[, question], levels = 1:5))), 2), "\n")
    fig <- plot_ly(
        x = c("Strongly disagree", "Disagree", "Neither agree nor agree", "Agree", "Strongly agree"),
        y = table(factor(survey_response_data[, question], levels = 1:5)),
        type = "bar",
        name = question_labels[question],
    ) %>%
    layout(title = "Hello")
    fig_list <- c(fig_list, list(fig))
}

subplot(fig_list, nrows = length(question_1_items), shareX = TRUE, shareY = TRUE) %>%
    layout(title = question_labels["Q_1"],
           xaxis = list(categoryorder = "array", categoryarray = c("Strongly disagree", "Disagree", "Neither agree nor agree", "Agree", "Strongly agree")))


In [ ]:
#Q2 responses
question_2_items <- c("Q2_1", "Q2_2", "Q2_3", "Q2_4", "Q2_5")
fig_list <- list()
for (question in question_2_items) {
    cat(question_labels[question],":\n")
    cat("Skewness: ", round(skewness(table(factor(survey_response_data[, question], levels = 1:4))), 2), "\n")
    cat("Kurtosis: ", round(kurtosis(table(factor(survey_response_data[, question], levels = 1:4))), 2), "\n")
    fig <- plot_ly(
        x = c("Very unhelpful", "Unhelpful", "Helpful", "Very helpful"),
        y = table(factor(survey_response_data[, question], levels = 1:4)),
        type = "bar",
        name = question_labels[question]
    )
    fig_list <- c(fig_list, list(fig))
}

subplot(fig_list, nrows = length(question_2_items), shareX = TRUE, shareY = TRUE) %>%
    layout(title = question_labels["Q_2"],
           xaxis = list(categoryorder = "array", categoryarray = c("Very unhelpful", "Unhelpful", "Helpful", "Very helpful")))


In [ ]:
#Q3 responses
questions_3_items <- c("Q3_1", "Q3_2")
fig_list <- list()
for (question in questions_3_items) {
    cat(question_labels[question],":\n")
    cat("Skewness: ", round(skewness(table(factor(survey_response_data[, question], levels = 1:5))), 2), "\n")
    cat("Kurtosis: ", round(kurtosis(table(factor(survey_response_data[, question], levels = 1:5))), 2), "\n")
    fig <- plot_ly(
        x = c("Strongly disagree", "Disagree", "Neither agree nor agree", "Agree", "Strongly agree"),
        y = table(factor(survey_response_data[, question], levels = 1:5)),
        type = "bar",
        name = question_labels[question])
    fig_list <- c(fig_list, list(fig))
}

subplot(fig_list, nrows = length(questions_3_items), shareX = TRUE, shareY = TRUE) %>%
    layout(title = question_labels["Q_3"],
            xaxis = list(categoryorder = "array", categoryarray = c("Strongly disagree", "Disagree", "Neither agree nor agree", "Agree", "Strongly agree")))


In [ ]:
#Q4 responses
cat("Skewness: ", round(skewness(table(factor(survey_response_data[, "Q4"], levels = 1:5))), 2), "\n")
cat("Kurtosis: ", round(kurtosis(table(factor(survey_response_data[, "Q4"], levels = 1:5))), 2), "\n")
fig <- plot_ly(
    x = c("Far too challenging", "A little too challenging", "A good level of difficulty", "A little too easy", "Far too easy"),
    y = table(factor(survey_response_data[, "Q4"], levels = 1:5)),
    type = "bar",
    name = question_labels["Q4"]) %>%
        layout(title = question_labels["Q4"],
            xaxis = list(categoryorder = "array", categoryarray = c("Far too challenging", "A little too challenging", "A good level of difficulty", "A little too easy", "Far too easy")))
fig

In [ ]:
#Q5 responses
cat("Skewness: ", round(skewness(table(factor(survey_response_data[, "Q5"], levels = 1:4))), 2), "\n")
cat("Kurtosis: ", round(kurtosis(table(factor(survey_response_data[, "Q5"], levels = 1:4))), 2), "\n")
fig <- plot_ly(
    x = c("Not at all", "I barely use it", "I sometimes use it", "I use it every time I get an error"),
    y = table(factor(survey_response_data[, "Q5"], levels = 1:4)),
    type = "bar",
    name = question_labels["Q5"]
) %>%
    layout(title = question_labels["Q5"],
           xaxis = list(categoryorder = "array", categoryarray = c("Not at all", "I barely use it", "I sometimes use it", "I use it every time I get an error")))
fig


In [ ]:
#Q6 responses
cat("Skewness: ", round(skewness(table(factor(survey_response_data[, "Q6"], levels = 1:4))), 2), "\n")
cat("Kurtosis: ", round(kurtosis(table(factor(survey_response_data[, "Q6"], levels = 1:4))), 2), "\n")
fig <- plot_ly(
    x = c("Very unhelpful", "Unhelpful", "Helpful", "Very helpful"),
    y = table(factor(survey_response_data[, "Q6"], levels = 1:4)),
    type = "bar",
    name = question_labels["Q6"]
) %>%
    layout(title = question_labels["Q6"],
           xaxis = list(categoryorder = "array", categoryarray = c("Very unhelpful", "Unhelpful", "Helpful", "Very helpful")))
fig


## Appendix B. Written Responses
The following code cell displays the written feedback that students gave in response to the optional free-text question "If there's anything else you'd like to mention about PRIMMDebug or SIFFT, please write it here."

In [ ]:
for (i in 1:nrow(survey_response_data)) {
    if (survey_response_data[i, "Q7"] != "") {
        print(paste(survey_response_data[i, "Q7"]))
    }
}

## Appendix C. Original FA Plan

We initially grouped the survey responses into some groups based on the content of the survey items:
- **Usability**: The usability of programs, borrowing items from the [System Usability Scale](https://en.wikipedia.org/wiki/System_usability_scale).
- **Utility: Restrictive Factors**: Perceived utility of the parts of the PRIMMDebug tool that restricted students' "programming autonomy".
- **Utility: PRIMMDebug Challenges**: Perceived utility of the PRIMMDebug challenges they attempted.
- **Utility: SIFFT**: Perceived utility of the SIFFT process that teachers taught for debugging.

Each of these constructs Likert scale response questions associated with them.